importing modules and files

In [1]:
import xml.etree.ElementTree as ET 
import re
import html
from Api.apiRequests import getJobListings
from operatingSys.servieces import importXml
from xml.sax.saxutils import unescape

Importing and providing the xml data from server to local, then parsing to be usable

In [2]:
url = 'http://localhost:8080/' #source XML address
xmlData = getJobListings(url)

In [3]:
xmlFolderAddress = "xmlFiles/fetchedXml.xml" # the target folder that we have imported our xml response

importXml(xmlFolderAddress,xmlData) #writes the fetched xml to desired address
root = ET.parse(xmlFolderAddress).getroot()

In [4]:
# to validate the RGB color
_rgbstring = re.compile(r'#[a-fA-F0-9]{6}$')

def isrgbcolor(value):
    return bool(_rgbstring.match(value))

In [5]:
import dominate
from dominate.tags import *

Extracting xml tags to variables:
ex:
    jobTitle = 'Python Entwickler '
    responsibilities = '...'

In [53]:
def colorDifiner(element,col):
    if isrgbcolor(col):
        element['style'] = "color : {color}".format(color=col)

In [52]:
imageUrl = ''
heading_color = ''
for joblisting in root:
    for i in (joblisting.iter('language')):
        if i.text == 'DE': 
            unique_job_id = joblisting.get('unique_job_id')
            for elem in joblisting.findall('job_data/jobtitle'):
                jobTitle = elem.text.replace('\n', '')
            for elem in joblisting.findall('job_data/layout_data/images/header'):
                imageUrl = elem.text
            for elem in joblisting.findall('job_data/company_data/companyname'):
                companyName = elem.text.replace('\n', '')
            
            # City list
            locationList = []
            for location in joblisting.findall('job_data/joblocations/location'):
                for cityName in joblisting.findall('job_data/joblocations/location/city'):
                    city = cityName.text
                    city = city.replace(' ', '')
                    city = city.replace('\n', '')

                    if len(joblisting.findall('job_data/joblocations/location/postalcode')) != 0 :
                        for location in location.iter('postalcode'):
                            postalCode = location.text
                            postalCode = postalCode.replace(' ', '')
                            postalCode = postalCode.replace('\n', '')
                            cityAndZip = postalCode +','+ city
                            locationList.append(cityAndZip)
                    else:
                            cityAndZip = city
                            locationList.append(cityAndZip)
                    
            # Contents
            for elem in joblisting.findall('job_data/introduction'):
                introduction = elem.text
                introductionTitle = elem.get('title')
            for elem in joblisting.findall('job_data/responsibilities'):
                responsibilities = elem.text
                responsibilitiesTitle = elem.get('title')
            for elem in joblisting.findall('job_data/qualifications'):
                qualifications = elem.text
                qualificationsTitle = elem.get('title')
            for elem in joblisting.findall('job_data/we_offer'):
                we_offer = elem.text
                we_offerTitle = elem.get('title')
            for elem in joblisting.findall('job_data/contact_data'):
                contact_data = elem.text
                contact_dataTitle = elem.get('title')
            # color
            for elem in joblisting.findall('job_data/layout_data/colors/heading_color'):
                heading_color = elem.text.replace('\n', '').replace(' ', '')
            for elem in joblisting.findall('job_data/application_procedure/url'):
                applyUrl = elem.text
            

            
            # making HTML files from above variables

            def create_page(
                unique_job_id,jobTitle,imageUrl,companyName,locationList,introduction,introductionTitle,responsibilities,responsibilitiesTitle,qualifications,qualificationsTitle,we_offer,we_offerTitle,contact_data,contact_dataTitle,heading_color,applyUrl
            ):
                doc = dominate.document(title = jobTitle)
                _div = doc.body.add(div(cls="joblisting" , id=unique_job_id))
                if len(imageUrl) > 0:
                    _image =_div.add(img(src=imageUrl))
                _h1 = _div.add(h1(jobTitle))
                _h2 = _div.add(h2(companyName))
                citiesAll = ''
                if len(locationList) > 1:
                    for city in locationList:
                        citiesAll = citiesAll + city + ','
                else:
                    citiesAll = locationList[0]
                _pCityNames = _div.add(p(citiesAll))
                _divContents = _div.add(div(cls="content_sections"))
                if introductionTitle :
                    h2IntroductionTitle = _divContents.add(h2(introductionTitle))
                    colorDifiner(h2IntroductionTitle,heading_color)
                _pIntroduction = _divContents.add(p(introduction))
                if responsibilitiesTitle :
                    _divContents.add(h2(responsibilitiesTitle))
                _pResponsibility = _divContents.add(p(responsibilities))
                if qualificationsTitle :
                    _divContents.add(h2(qualificationsTitle))
                _pQualificcations = _divContents.add(p(qualifications))
                if we_offerTitle :
                    _divContents.add(h2(we_offerTitle))
                _pWe_offer = _divContents.add(p(we_offer))
                if contact_dataTitle :
                    _divContents.add(h2(contact_dataTitle))
                _pContact_data = _divContents.add(p(contact_data))
                
                
                HtmlFileAddress = "outputs/id_{address}.html".format(address=unique_job_id)

                # doc = str(doc).replace('&lt;', '<').replace('&gt;', '>').replace('&amp;', '&')
                doc = html.unescape(str(doc))
                with open(HtmlFileAddress,'a') as f:
                    # delete the old xml content
                    f.seek(0)  # sets  point at the beginning of the file
                    f.truncate()  # Clear previous content
                    # writing new data
                    f.write(str(doc))
                    f.close() # Close file
            create_page(
                unique_job_id,jobTitle,imageUrl,companyName,locationList,introduction,introductionTitle,responsibilities,responsibilitiesTitle,qualifications,qualificationsTitle,we_offer,we_offerTitle,contact_data,contact_dataTitle,heading_color,applyUrl
            )